In [4]:
import pandas as pd
import numpy as np
import glob
import warnings
warnings.filterwarnings("ignore")

# Reading/Cleaning Data

In [5]:
# The filepath needs to be where the files are located on your device
file_paths = glob.glob('/Users/jeremylee/Desktop/CS-Projects/T4SG/21su-usda-app-rationalization/data/*.xlsx')

In [6]:
df = pd.DataFrame()
for filepath in file_paths:
    df = pd.concat([df, pd.read_excel(filepath)])

In [ ]:
drop_columns = ["AD_Site_Name0", "User", "Agency"]

df.drop(columns=drop_columns, inplace=True)

In [ ]:
### Remove all the rows of data from Microsoft servers, as we’re only interested in workstations

df['C054'].fillna('',inplace=True)  # replaces NaN values in column C054 with ‘’ nothing, needed for next segment of code to work

index = df[df['C054'].str.lower().str.contains('server')].index # creates an index of rows containing server in the OS column (C054)

df.drop(index, inplace=True)  # this deletes all rows where the word 'server' is in column C054, from prior line's index

In [ ]:
### Removing GOTS applications

df = df[df["Publisher"].str.lower().str.contains("usda") == False]

# Filtering out utility applications

In [ ]:
utility_keywords = ["driver", "update", "compiler", "decompiler", 
                    "installer", "utility", "plugin", "tool"]

df["Application"].fillna('',inplace=True)
df["Utility"] = 0
for keyword in utility_keywords:
    df.loc[df["Application"].str.lower().str.contains(keyword, na=False), "Utility"] = 1

In [ ]:
utility_publishers = ["Intel", "Intel Corporation", "Intel(R) Corporation",
                      "Advanced Micro Devices, Inc.", "Advanced Micro Devices Inc.", "AMD",
                      "Dell", "Dell Inc.", "Dell, Inc.", "Dell Inc"]

df["Publisher"].fillna('',inplace=True)
df.loc[df.Publisher.isin(utility_publishers), "Utility"] = 1

In [ ]:
# Microsoft and Microsoft Corporation are the same

df.loc[df["Publisher"] == "Microsoft Corporation", "Publisher"] = "Microsoft" 

In [ ]:
utility_list = df[df["Utility"] == 1].drop(columns="Utility")
utility_list = utility_list[["Publisher", "Application"]].drop_duplicates()
utility_df = pd.DataFrame(utility_list)

In [ ]:
counts = []
value_counts = df.groupby("Application").count()["System Name"]
for app in utility_df["Application"]:
    counts.append(value_counts[app])
utility_df["Count"] = counts
utility_df.sort_values(by="Count", ascending=False, inplace=True)

In [ ]:
single_workstation_utilities = np.array(utility_df.loc[utility_df["Count"] == 1, "Application"])
single_utilities = df[df["Application"].isin(single_workstation_utilities)][["System Name", "Publisher", "Application"]]

In [ ]:
single_utilities.to_csv("data/single_workstation_utilities.csv")

In [ ]:
utility_df.to_csv("data/utilities.csv")

In [ ]:
df.to_csv("data/flagged_utilities.csv")

# Categorizing Business Applications

In [ ]:
main_df = df[df["Utility"] == 0].drop(columns="Utility")

In [ ]:
unique_apps = main_df["Application"].unique()

# Getting Information on Microsoft Updates

In [ ]:
microsoft_updates = utility_df.loc[utility_df["Application"].str.contains("Update for Microsoft Office"), "Application"]

In [ ]:
def get_version(s):
    for i in range(len(s) - 3):
        if s[i:i+4].isnumeric():
            return s[i:i+4]

In [ ]:
microsoft_updates_df = pd.DataFrame()
microsoft_updates_df["Application"] = utility_df.loc[utility_df["Application"].str.contains("Update for Microsoft Office"), "Application"]
microsoft_updates_df["Count"] = utility_df.loc[utility_df["Application"].str.contains("Update for Microsoft Office"), "Count"]

In [ ]:
proportion = []
percentage = []

update_count = 0
for update in microsoft_updates:
    workstations = df.loc[df["Application"] == update, "System Name"]
    count = 0
    for workstation in workstations:
        workstation_apps = main_df.loc[main_df["System Name"] == workstation, "Application"]
        office_apps = workstation_apps[workstation_apps.str.contains("Microsoft Office")]
        for app in office_apps:
            if get_version(update) in app:
                count += 1
                break
    proportion.append(str(count) + "/" + str(len(workstations)))
    percentage.append(count / len(workstations))
    update_count += 1
    
microsoft_updates_df["Proportion of Workstations with Office Installed"] = proportion
microsoft_updates_df["Percentage of Workstations with Office Installed"] = percentage

In [159]:
microsoft_updates_df["Percentage of Workstations with Office Installed"] *= 100

In [161]:
microsoft_updates_df.to_csv("microsoft_updates.csv")

# Getting Sample of 20 Apps for USDA to Categorize

In [167]:
top20_apps = main_df["Application"].value_counts().rename_axis('Application').reset_index(name='Count')

In [170]:
top20_apps.head(20).to_csv("sample20_applications.csv")

# Normalizing App Names

In [21]:
# Getting rid of versions

business_apps = list(main_df.Application.unique())

counts = []
value_counts = main_df.groupby("Application").count()["System Name"]
for app in business_apps:
    counts.append(value_counts[app])

normalized_df = pd.DataFrame()
normalized_df["old_name"] = business_apps
normalized_df["new_name"] = business_apps
normalized_df["count"] = counts

normalized_df.sort_values(by="count", ascending=False, inplace=True)

In [22]:
def is_non_number(s):
    if len(s) == 0:
        return False
    if s[0].lower() == "v":
        for char in s[1:]:
            if char.isalpha():
                return True
        return False
    else:
        for char in s:
            if char.isalpha():
                return True
        return False

def remove_comma(word):
    if len(word) == 0:
        return word
    if word[-1] == ",":
        return word[:-1]
    return word

tags = ["ARM64", "arm64", "amd64", "arm", "ARM",
        "X64", "X86", "x64", "x86", "64-bit", "32-bit", "32bit", "64bit"]

def remove_tag_word(word):
    new_word = word
    for tag in tags:
        if "(" + tag + ")" in new_word:
            new_word = new_word.replace("(" + tag + ")", "")
        elif "_" + tag in new_word:
            new_word = new_word.replace("_" + tag, "")
        elif tag in new_word:
            new_word = word.replace(tag, "")
        
    return new_word

def remove_parentheses_word(word):
    if len(word) == 0:
        return word
    if (word[0] == "(" and ")" not in word) or (word[-1] == ")" and "(" not in word):
        return ""
    return word

def remove_version_word(word):
    if "version" in word.lower():
        return False
    return True

def remove_blank_words(word):
    return word != ""

def normalize_name(app_name):
    words = str(app_name).split()
    
    words = list(filter(is_non_number, words)) # Removing version and years
    words = list(map(remove_comma, words)) # Removing ending commas
    words = list(map(remove_tag_word, words)) # Removing 32 vs 64 bit
    words = list(map(remove_parentheses_word, words)) # Removing parentheses
    words = list(filter(remove_version_word, words)) # Removing the word "version"
    words = list(filter(remove_blank_words, words)) # Removing blank words
    
    return " ".join(words)

normalized_df["new_name"] = normalized_df.old_name.apply(normalize_name)

In [23]:
normalized_df.to_csv("data/normalized_apps.csv")